#### Play with mat strucuture
In model data, we are mostly interested in mod_struc['S']
1) S; 

2) BkgdConc, Chem, Cnames, Conc, InitConc, Met, ModelOptions, SolarParam, StepIndex, Time, iRO2

3) Names in MCM/GEOS-Chem format

dir(mat_struc['S']['Conc'])

**In observation data, we still need to look into it when we further come back to office.**

**Bkgdconc (20*18)**

18: default + names of 17 trace gases?

what's 20?

sec item to acess to the data

**It proves that the sec test we reach the data we need.**

**We assume the order is same in the conc and in the cnames**

In [1]:
# import modules
import scipy.io
import numpy as np
import pandas as pd

In [2]:
# change working directory
import os
os.chdir('/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/Collaborators/Wade/output_data/')
#os.chdir('/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/Furans/output_data/')

In [3]:
# design function to load table variable from MAT file.
def getF0AM_mat_bkg(F0AM_model_file, variable_name):
    # load file
    mod_struc = scipy.io.loadmat(F0AM_model_file)
    # get concentration and bkgdconc from model
    mod_bkgdconc = (mod_struc['S']['BkgdConc']).item().item()
    # convert tuple into array
    mod_bkgdconc = np.array(mod_bkgdconc)
    # convert 3D into 2D
    mod_bkgdconc = mod_bkgdconc[:,:,0]
    # get names of bkg values
    mod_bkgdconc_names = list((mod_struc['S']['BkgdConc']).item().dtype.names)
    # make the data frame
    mod_bkgconc_df = pd.DataFrame(mod_bkgdconc, index=mod_bkgdconc_names)
    mod_bkgconc_df.dropna(how='all')
    return mod_bkgconc_df.loc[variable_name,:][0]
    

In [4]:
# design function to load table variable from MAT file.
# return specific variable from the FOAM output
def getF0AM_mat_conc(F0AM_model_file, variable_name):
    """
    read a struct-ified table variable (and column names) from a MAT-file
    and return pandas.DataFrame object.
    """
    # load file
    mod_struc = scipy.io.loadmat(F0AM_model_file)
    # get concentration from model
    mod_conc = (mod_struc['S']['Conc']).item().item()
    # get cnames
    mod_cnames = mod_struc['S']['Cnames'].item()
    # get the mdoel time (in sec)
    mod_time = mod_struc['S']['Time'].item()
    # convert tuple into array
    mod_conc = np.array(mod_conc)
    # convert 3D into 2D
    mod_conc = mod_conc[:,:,0]
    # make the data frame
    mod_df = pd.DataFrame(np.hstack((mod_conc, mod_cnames)))
    # save last column of dataframe as a series
    last_column = mod_df.iloc[: , -1]
    # delete the last column
    mod_df = mod_df.iloc[:, :-1]
    # get names from the model
    mod_cnames = []
    for data in last_column:
        data = data.item()
        mod_cnames.append(data)
    # rename index of the python dataframe
    mod_df.index = mod_cnames
    # rename the column names as time
    mod_time = mod_time[:,0]
    mod_df.columns = mod_time
    return mod_df.loc[variable_name,:]
 

##### For Wade

In [5]:
# Modify from the getF0AM_mat_conc function above
# 1) return concentrations; 2) return CO concentrations
# load Concentration data, customized

File_input = 'MCMv331_baseRF03_times3'
#File_input = 'MCMv331_baseRF03_times1'

#File_input = 'GCv13_baseRF03'
mod_struc = scipy.io.loadmat(File_input + '.mat')
# get concentration from model
mod_conc = (mod_struc['S']['Conc']).item().item()
# get cnames
mod_cnames = mod_struc['S']['Cnames'].item()
# get the mdoel time (in sec)
mod_time = mod_struc['S']['Time'].item()
# convert tuple into array
mod_conc = np.array(mod_conc)
# convert 3D into 2D
mod_conc = mod_conc[:,:,0]
# make the data frame
mod_df = pd.DataFrame(np.hstack((mod_conc, mod_cnames)))
# save last column of dataframe as a series
last_column = mod_df.iloc[: , -1]
# delete the last column
mod_df = mod_df.iloc[:, :-1]
# get names from the model
mod_cnames = []
for data in last_column:
    data = data.item()
    mod_cnames.append(data)
# rename index of the python dataframe
mod_df.index = mod_cnames
# rename the column names as time
mod_time = mod_time[:,0]
mod_df.columns = mod_time
# save out data for Wade
mod_df.transpose().iloc[: , 1:].to_csv(File_input + '.csv', index=True)

# Do the calcluation 
'''
#Lnames (101 compounds) 
#Loss (3169 datapoints, 101 compounds) # unit: ppt/s
#OH (3169 datapoints) # unit: unknown, assuming it's ppt?

# load RF file, customized
mod_struc_OH = scipy.io.loadmat('./pltrate_RF' + RF_num + '.mat')
Pnames = mod_struc_OH['Pnames']
Prod = mod_struc_OH['Prod']
Lnames = mod_struc_OH['Lnames'][:-1]# remove dilution
Lnames = np.concatenate(Lnames.flatten().tolist()).tolist()
Loss = mod_struc_OH['Loss']

# calculate OHr
for nn in range(len(Lnames)):
    Loss[:, nn] = -Loss[:, nn]/mod_df.loc['OH'].values

# save out the OH reactivity data
df_OHr = pd.DataFrame(Loss, 
                      columns=np.array(Lnames),
                      index = mod_time)
df_OHr.to_csv('./pltrate_RF' + RF_num + '.csv', index=True)

# save out the OH reacitivity data + CO concentration data
result = pd.concat([df_OHr, mod_df.transpose().iloc[: , 1:]['CO']], axis=1)
result.to_csv('./OHr_CO_RF' + RF_num + '.csv', index=True)
'''

"\n#Lnames (101 compounds) \n#Loss (3169 datapoints, 101 compounds) # unit: ppt/s\n#OH (3169 datapoints) # unit: unknown, assuming it's ppt?\n\n# load RF file, customized\nmod_struc_OH = scipy.io.loadmat('./pltrate_RF' + RF_num + '.mat')\nPnames = mod_struc_OH['Pnames']\nProd = mod_struc_OH['Prod']\nLnames = mod_struc_OH['Lnames'][:-1]# remove dilution\nLnames = np.concatenate(Lnames.flatten().tolist()).tolist()\nLoss = mod_struc_OH['Loss']\n\n# calculate OHr\nfor nn in range(len(Lnames)):\n    Loss[:, nn] = -Loss[:, nn]/mod_df.loc['OH'].values\n\n# save out the OH reactivity data\ndf_OHr = pd.DataFrame(Loss, \n                      columns=np.array(Lnames),\n                      index = mod_time)\ndf_OHr.to_csv('./pltrate_RF' + RF_num + '.csv', index=True)\n\n# save out the OH reacitivity data + CO concentration data\nresult = pd.concat([df_OHr, mod_df.transpose().iloc[: , 1:]['CO']], axis=1)\nresult.to_csv('./OHr_CO_RF' + RF_num + '.csv', index=True)\n"

In [7]:
test = mod_df.transpose().iloc[: , 1:]

In [9]:
test.head(2)

,RO2,C918NO3,C134CO,NBUTACET,C124O2,PROL1MPAN,BCCOOH,IPRHOCO2H,TM124NO2O,C3EODBCO2H,...,lumpdummy21,lumpdummy22,lumpdummy23,lumpdummy24,lumpdummy25,lumpdummy26,lumpdummy27,lumpdummy28,lumpdummy29,lumpdummy30
0.000000e+00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6.650500e-12,4.43699e-13,4.27425e-62,1.03883e-143,0,0,0,4.03039e-30,9.68859e-74,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
test.columns

Index(['RO2', 'C918NO3', 'C134CO', 'NBUTACET', 'C124O2', 'PROL1MPAN', 'BCCOOH',
       'IPRHOCO2H', 'TM124NO2O', 'C3EODBCO2H',
       ...
       'lumpdummy21', 'lumpdummy22', 'lumpdummy23', 'lumpdummy24',
       'lumpdummy25', 'lumpdummy26', 'lumpdummy27', 'lumpdummy28',
       'lumpdummy29', 'lumpdummy30'],
      dtype='object', length=6830)

In [15]:
sum(test['dummy1'])

0.0

In [13]:
species = test.columns
for spec in species:
    if 'dummy' in spec:
        print(spec)

dummy1
dummy2
dummy3
dummy4
dummy5
dummy6
dummy7
dummy8
dummy9
dummy10
dummy11
dummy12
dummy13
dummy14
dummy15
dummy16
dummy17
dummy18
dummy19
dummy20
dummy21
dummy22
dummy23
dummy24
dummy25
dummy26
dummy27
dummy28
dummy29
dummy30
dummy31
dummy32
dummy33
dummy34
dummy35
dummy36
dummy37
dummy38
dummy39
dummy40
dummy41
dummy42
dummy43
dummy44
dummy45
dummy46
dummy47
dummy48
dummy49
dummy50
dummy51
dummy52
dummy53
dummy54
dummy55
dummy56
dummy57
dummy58
dummy59
dummy60
dummy61
dummy62
dummy63
dummy64
dummy65
dummy66
dummy67
dummy68
dummy69
dummy70
dummy71
dummy72
dummy73
dummy74
dummy75
dummy76
dummy77
dummy78
dummy79
dummy80
dummy81
dummy82
dummy83
dummy84
dummy85
dummy86
dummy87
dummy88
dummy89
dummy90
dummy91
dummy92
dummy93
dummy94
dummy95
dummy96
dummy97
dummy98
dummy99
dummy100
dummy101
dummy102
dummy103
dummy104
dummy105
dummy106
dummy107
dummy108
dummy109
dummy110
dummy111
dummy112
dummy113
dummy114
dummy115
dummy116
dummy117
dummy118
dummy119
dummy120
lumpdummy1
lumpdummy2
lumpdu